# Install and Download

In [1]:
#Install pyrwr
!git clone https://github.com/jinhongjung/pyrwr.git
%cd ./pyrwr
!pip install -r requirements.txt

Cloning into 'pyrwr'...
remote: Enumerating objects: 305, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 305 (delta 68), reused 92 (delta 57), pack-reused 195
Receiving objects: 100% (305/305), 193.61 KiB | 19.36 MiB/s, done.
Resolving deltas: 100% (185/185), done.
/home/user/Desktop/lexas/pyrwr
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 78 kB 15.3 MB/s            
     |████████████████████████████████| 87 kB 28.6 MB/s            
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement numpy~=1.22.4 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 1.13.3, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 

In [3]:
#Downloading data
#FunCoup5
!wget "https://funcoup5.scilifelab.se/downloads/download.action?type=network&instanceID=24480085&fileName=FC5.0_H.sapiens_full.gz" -O funcoup5.tab.gz
!gunzip funcoup5.tab.gz

#STRINGv10
!wget http://version10.string-db.org/download/protein.links.v10/9606.protein.links.v10.txt.gz -O ./protein.links.v10.txt.gz
!gunzip ./protein.links.v10.txt.gz

--2024-01-09 21:09:42--  https://funcoup5.scilifelab.se/downloads/download.action?type=network&instanceID=24480085&fileName=FC5.0_H.sapiens_full.gz
funcoup5.scilifelab.se (funcoup5.scilifelab.se) をDNSに問いあわせています... 130.237.48.106
funcoup5.scilifelab.se (funcoup5.scilifelab.se)|130.237.48.106|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 301 Moved Permanently
場所: https://funcoup.org/downloads/download.action?type=network&instanceID=24480085&fileName=FC5.0_H.sapiens_full.gz [続く]
--2024-01-09 21:09:45--  https://funcoup.org/downloads/download.action?type=network&instanceID=24480085&fileName=FC5.0_H.sapiens_full.gz
funcoup.org (funcoup.org) をDNSに問いあわせています... 130.237.48.106
funcoup.org (funcoup.org)|130.237.48.106|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 
長さ: 284028906 (271M) [application/octet-stream]
`funcoup5.tab.gz' に保存中

funcoup5.tab.gz     100%[===================>] 270.87M  11.8MB/s    時間 27s     

2024-01-09 21:10:14 (10.1 MB/s) - `funcoup5.tab.gz' へ保

In [40]:
import json
#Dictionaries for ID conversion
path = "../data/ensg.json"
with open(path, mode="r") as f:
    sym_to_ensg = json.load(f)
ensg_to_sym = {value: key for key, value in sym_to_ensg.items()}

path = "../data/ensp.json"
with open(path, mode="r") as f:
    sym_to_ensp = json.load(f)
ensp_to_sym = {value: key for key, value in sym_to_ensp.items()}

# FunCoup

Comparison is performed using the code in Evaluation.ipynb

In [68]:
import tqdm

# Create a mapping of genes to unique numbers from the funcoup file
gene_list, numbd =[], {}
c = 0

with open("funcoup5.tab", "r") as f:
    next(f)  # Skip header
    for line in tqdm.tqdm(f):
        gene1, gene2 = line.split("\t")[2:4]
        if gene1 not in numbd:
            numbd[gene1] = str(c)
            gene_list.append(gene1)
            c += 1
        if gene2 not in numbd:
            numbd[gene2] = str(c)
            gene_list.append(gene2)
            c += 1

5036826it [00:06, 746856.31it/s]


In [5]:
import tqdm

# Paths
input_path = "funcoup5.tab"
output_path = "funcoup_graph.tsv"

# Mapping genes to unique numbers and writing relationships to a new file
with open(input_path, "r") as input_file, open(output_path, "w") as output_file:
    next(input_file)  # Skip header
    for line in tqdm.tqdm(input_file):
        gene1, gene2, score = line.strip().split("\t")[2], line.strip().split("\t")[3], line.strip().split("\t")[0]
        output_file.write("\t".join([numbd[gene1], numbd[gene2], score]) + "\n")


5036826it [00:16, 297759.40it/s]


In [70]:
from pyrwr.rwr import RWR

# Set up and read graph
rwr = RWR()
rwr.read_graph("./funcoup_graph.tsv", "undirected")

#Using a sample gene ID for testing
seed = 0
result = rwr.compute(seed, c=0.15, epsilon=1e-9, max_iters=100, device="cpu") #Default parameter settings
result

The iteration has converged at 73-iter:  73%|███████▎  | 73/100 [00:00<00:00, 100.23it/s]


array([1.51652320e-01, 3.31746339e-04, 8.58316114e-04, ...,
       1.13060100e-08, 2.51885949e-08, 7.81477495e-09])

In [120]:
# Calculate RWR similarity for comparison
import pandas as pd
import numpy as np
symbols = ["PLK4","SASS6","CEP152","CEP192","CEP63","PCNT"] #For demonstration

for symbol in symbols:
    ensg =sym_to_ensg[symbol]
    seed = numbd[ensg]
    result = rwr.compute(int(seed), c=0.15, epsilon=1e-9, max_iters=100, device="cpu")

    df = pd.DataFrame({"ENSG":gene_list,"Score":result})
    df["Symbol"] = [ensg_to_sym[gene] if gene in ensg_to_sym else None for gene in gene_list]
    df = df.reindex(columns=['Symbol', 'Score', 'ENSG'])
    df.dropna().to_csv(f"../result/funcoup_rwr/{symbol}.csv",index=None)

#Evaluation is performed in Evaluation.ipynb

The iteration has converged at 77-iter:  77%|███████▋  | 77/100 [00:00<00:00, 103.55it/s]


# STRING

Comparison was performed using the code in Evaluation.ipynb

In [63]:
import tqdm

# Create a mapping of proteins to unique numbers from the file
protein_list, numbd2 = [], {}
c = 0

with open("./protein.links.v10.txt", "r") as f:
    next(f)  # Skip header
    for line in tqdm.tqdm(f):
        protein1, protein2 = line.strip().split(" ")[:2]
        if protein1 not in numbd2:
            numbd2[protein1] = str(c)
            protein_list.append(protein1)
            c += 1
        if protein2 not in numbd2:
            numbd2[protein2] = str(c)
            protein_list.append(protein2)
            c += 1

8548002it [00:05, 1660975.29it/s]


In [65]:
import tqdm

#Paths
input_path = "./protein.links.v10.txt"
output_path = "string_graph.tsv"

# Mapping genes to unique numbers and writing relationships to a new file
with open(input_path, "r") as input_file, open(output_path, "w") as output_file:
    next(input_file)  # Skip header
    for line in tqdm.tqdm(input_file):
        gene1, gene2, score = line.strip().split(" ")
        output_file.write(f"{numbd2[gene1]}\t{numbd2[gene2]}\t{score}\n")

8548002it [00:06, 1312732.39it/s]


In [72]:
from pyrwr.rwr import RWR

# Set up and read graph
rwr2 = RWR()
rwr2.read_graph("./string_graph.tsv", "undirected")

#Using a sample gene ID for testing
seed = 0
result = rwr2.compute(seed, c=0.15, epsilon=1e-9, max_iters=100, device="cpu") #Default parameter settings
result

The iteration has converged at 78-iter:  78%|███████▊  | 78/100 [00:00<00:00, 114.47it/s]


array([1.50372212e-01, 2.13577422e-04, 1.02885063e-04, ...,
       4.56252531e-08, 3.27692446e-08, 3.11163359e-08])

In [117]:
# Calculate RWR similarity for comparison

import pandas as pd
import numpy as np
symbols = ["PLK4","SASS6","CEP152","CEP192","PCNT"] #For demonstration

with open(output_path, "w") as f:
    for symbol in symbols:
        ensp =sym_to_ensp[symbol]
        seed = numbd2["9606."+ensp]
        result = rwr2.compute(int(seed), c=0.15, epsilon=1e-9, max_iters=100, device="cpu")
        
        df = pd.DataFrame({"ENSP":protein_list,"Score":result})
        df["Symbol"] = [ensp_to_sym[prot[5:]] if prot[5:] in ensp_to_sym else None for prot in protein_list]
        df = df.reindex(columns=['Symbol', 'Score', 'ENSP'])
        df.dropna().to_csv(f"../result/string_rwr/{symbol}.csv",index=None)

#Evaluation is performed in Evaluation.ipynb

The iteration has converged at 78-iter:  78%|███████▊  | 78/100 [00:00<00:00, 110.05it/s]


# For training LEXAS-plus (with cutoffs)

In [ ]:
# Calculate RWR similarity for preparing data for training LEXAS-plus
import numpy as np
output_path = "funcoup_rwr.tsv"
cutoff_score = 0.001  # Use a threshold due to vast number of gene combinations
with open(output_path, "w") as f:
    for ensg, seed in numbd.items():
        result = rwr.compute(int(seed), c=0.15, epsilon=1e-9, max_iters=100, device="cpu")

        # Filter by score, exclude self-relations, and write to file
        for n, score in enumerate(result):
            if score > cutoff_score and ensg != gene_list[n]:
                f.write(f"{ensg}\t{gene_list[n]}\t{round(score*100,3)}\n")

        break # This is for testing, processing just one gene. Remove for processing all genes.

In [ ]:
# Calculate RWR similarity
import numpy as np

output_path = "string_rwr.tsv"
cutoff_score = 0.0005  # Use a threshold due to vast number of gene combinations

with open(output_path, "w") as f:
    for ensp, seed in numbd2.items():
        result = rwr.compute(int(seed), c=0.15, epsilon=1e-9, max_iters=100, device="cpu")

        # Filter by score, exclude self-relations, and write to file
        for n, score in enumerate(result):
            if score > cutoff_score and ensg != gene_list[n]:
                f.write(f"{ensp}\t{protein_list[n]}\t{round(score*100,3)}\n")

        break # This is for testing, processing just one gene. Remove for processing all genes.

# STRING-raw and FunCoup-raw
Evaluation is performed in Evaluation.ipynb

In [93]:
funcoup5_dict = {symbol:{} for symbol in sym_to_ensg}
with open("funcoup5.tab", "r") as f:
    next(f)  # Skip header
    for line in tqdm.tqdm(f):
        gene1, gene2, score = line.strip().split("\t")[2], line.strip().split("\t")[3], line.strip().split("\t")[0]
        try:
            symbol1,symbol2 = ensg_to_sym[gene1],ensg_to_sym[gene2]
        except KeyError:
            continue
        funcoup5_dict[symbol1][symbol2] = score
        funcoup5_dict[symbol2][symbol1] = score

5036826it [00:18, 275749.98it/s]


In [116]:
symbols = ["PLK4","SASS6","CEP152","CEP192","PCNT"]
for symbol in symbols:
    df = pd.DataFrame({"Symbol":funcoup5_dict[symbol].keys(),
                       "Score":funcoup5_dict[symbol].values()})
    df.dropna().to_csv(f"../result/funcoup_raw/{symbol}.csv",index=None)

In [111]:
string_dict = {symbol:{} for symbol in sym_to_ensp}
with open("./protein.links.v10.txt", "r") as f:
    next(f)  # Skip header
    for line in tqdm.tqdm(f):
        gene1, gene2, score = line.strip().split(" ")
        try:
            symbol1,symbol2 = ensp_to_sym[gene1[5:]],ensp_to_sym[gene2[5:]]
        except KeyError:
            continue
        string_dict[symbol1][symbol2] = score
        string_dict[symbol2][symbol1] = score

8548002it [00:09, 894341.09it/s]


In [113]:
symbols = ["PLK4","SASS6","CEP152","CEP192","PCNT"]
for symbol in symbols:
    df = pd.DataFrame({"Symbol":string_dict[symbol].keys(),
                       "Score":string_dict[symbol].values()})
    df.dropna().to_csv(f"../result/string_raw/{symbol}.csv",index=None)